In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
# !pip install stable-baselines3[extra]

In [1]:
import gym 
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import time

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        time.sleep(0.01)
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:27.0
Episode:2 Score:21.0
Episode:3 Score:23.0
Episode:4 Score:13.0
Episode:5 Score:14.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [35]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [36]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([4.2446265e+00, 5.7806556e+37, 1.3365486e-01, 3.3396322e+38],
      dtype=float32)

In [37]:
log_path = os.path.join('Training', 'Logs')

# 3. Train an RL Model

In [38]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [39]:
model.learn(total_timesteps=20000)

2022-04-10 22:47:30.629242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dheeraj/anaconda3/envs/RLp/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-10 22:47:30.629268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 1217 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1305        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008054259 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 7.96e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.56        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0193     |
|    value_loss           | 55.3        |
-----------------------------------------
---

# 4. Save and Reload Model

In [40]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [41]:
model.save(PPO_path)

In [42]:
del model

In [43]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [44]:
from stable_baselines3.common.evaluation import evaluate_policy

In [45]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

/home/dheeraj/anaconda3/envs/RLp/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


# 5. Test Model

In [46]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break
env.close()

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.03529671,  0.3391165 , -0.02472032,  0.09304576], dtype=float32)}]


# 6. Viewing Logs in Tensorboard

In [48]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [49]:
!tensorboard --logdir={training_log_path}

2022-04-10 22:50:50.382264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dheeraj/anaconda3/envs/RLp/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-10 22:50:50.382288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-10 22:50:51.397191: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-10 22:50:51.397224: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dheeraj-Legion-5-Pro-16ACH6H): /proc/driver/nvidia/version does not exist

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:


# 7. Adding a callback to the training Stage

In [50]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [51]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [52]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [53]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [54]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [55]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 3243 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 2120       |
|    iterations           | 2          |
|    time_elapsed         | 1          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00903296 |
|    clip_fraction        | 0.093      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | -0.0027    |
|    learning_rate        | 0.0003     |
|    loss                 | 6.2        |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0137    |
|    value_loss           | 47.2       |
----------------------------------------
---------------------

In [56]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [59]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

# 8. Changing Policies

In [60]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [61]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [62]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 2784 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1457        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014132254 |
|    clip_fraction        | 0.211       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00775    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.07        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0257     |
|    value_loss           | 20.7        |
-----------------------------------------
----------------------------------

# 9. Using an Alternate Algorithm

In [63]:
from stable_baselines3 import DQN

In [64]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [65]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.923    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 9613     |
|    time_elapsed     | 0        |
|    total_timesteps  | 162      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.887    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9667     |
|    time_elapsed     | 0        |
|    total_timesteps  | 237      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.858    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 9188     |
|    time_elapsed     | 0        |
|    total_timesteps  | 298      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 11458    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2595     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 11390    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2664     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 11201    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2808     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 10622    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4860     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 10593    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4930     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 10627    |
|    time_elapsed     | 0        |
|    total_timesteps  | 5016     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 10873    |
|    time_elapsed     | 0        |
|    total_timesteps  | 6904     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 10934    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7044     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 10963    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7147     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 10833    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9304     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 10833    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9372     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 10844    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9490     |
----------------------------------
Eval num_timesteps=9520, episode_reward=9.20 +/- 0.75
E

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 10562    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11399    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 10575    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11500    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 10566    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11555    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 10364    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13739    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 10352    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13831    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 10343    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13901    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 10244    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16081    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 10247    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16173    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 10237    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16308    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 10264    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18442    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 10264    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18511    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 10263    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18586    |
----------------------------------
----------------------------------
| rollout/          

In [66]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [67]:
model.save(dqn_path)

In [68]:
model = DQN.load(dqn_path, env=env)

In [71]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()